In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Concatenate, UpSampling2D, MaxPool2D, Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow.keras.backend as K

from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler, Normalizer
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

from codes.utils.HAR_utils import get_activity_index, plot_confusion_matrix, get_Train_Test, timeseries_standardize
from codes.models.attention import Attention2D

>>> Done:
>>> Done: Succesfully imported Utils module
>>> Done: Succesfully imported Utils module


## Transform Data

In [3]:
def get_train_test_bird(in_df,shuffle=False,trn='A'):
    
    in_df.dropna(inplace=True)
    birds = list(in_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
    max_len = 0

    for bird in birds:
        c_len = len(in_df[in_df['animal_tag'] == bird])
        
        if c_len > max_len:
            print(bird,' : ',c_len,' > ',max_len,' ? ',c_len > max_len)
            max_len = c_len
        print('bird: ',bird,'lenght: ',c_len)
    reqlen = int((int(max_len/512)+1)*512)
    print('reqlen: ',reqlen)

    features = in_df.columns[2:-2]
    X = []
    Y = []
    times = []
    normalizer = Normalizer()

    for bird in birds:
            x = np.array(in_df[in_df['animal_tag'] == bird][features].values)
            #print('x.shape',x.shape)
            xz = np.zeros((reqlen-len(x),len(features)))
            xs = normalizer.fit_transform(x)
            sx = np.concatenate((x,xz))
    
            y = np.array(in_df[in_df['animal_tag'] == bird]['l_val'].values)
            z = np.array([10]*(reqlen-len(y)))
            sy = np.concatenate((y,z))
    
            t = np.array(in_df[in_df['animal_tag'] == bird]['timestamp'].values)
            st = np.concatenate((t,z))
    
            X.append([sx])
            Y.append([sy])
            times.append(st)

    if trn != 1000:
        i=0
        if shuffle == True:
            shuffle == False
            print("Warning: shuffle cannot be used when selecting a specific bird")
        for bird in birds:
            if bird == trn:
                selector = i
            i=1+i
    else:
        selector = random.randint(0,len(birds)-1)

    if shuffle:
        p = list(zip(X,Y))
        random.shuffle(p)
        X, Y = zip(*p)
        X = list(X)
        Y = list(Y)

    X = np.array(X)
    Y = np.array(Y)

    times = np.array(times)
    print('X(shape): ',X.shape)
    print('Y(shape): ',Y.shape)
    print('times(shape): ',times.shape)

    testX = np.array([X[selector]])
    testY = np.array([Y[selector]])
    testtimes=times[selector]
    trainX = np.delete(X,selector,0)
    trainY = np.delete(Y,selector,0)
    
    return trainX,testX,trainY,testY,testtimes,selector

# Network design

In [4]:
def down_block(x, filters, kernel_size=(1, 3), padding="same", strides=1, name="down_"):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    p = MaxPool2D((1, 2),name=name+"M")(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(1, 3), padding="same", strides=1, name="up_"):
    us = UpSampling2D((1, 2),name=name+"U")(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"1")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name=name+"2")(c)
    return c

def bottleneck(x, filters, kernel_size=(1, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl1")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu",name="btl2")(c)
    c = Dropout(0.5)(c)
    return c

def build_unet_model(height,channels,class_num,f=32):
    
    inputs = Input((1, height, channels))
    
    p0 = inputs
    c1, p1 = down_block(p0, f, name="down_1_")
    c2, p2 = down_block(p1, f*2, name="down_2_")
    c3, p3 = down_block(p2, f*4, name="down_3_")
    p3 = Dropout(0.5)(p3)
    
    bn = bottleneck(p3, f*8)
    
    u1 = up_block(bn, c3, f*4, name="up_1_")
    u2 = up_block(u1, c2, f*2, name="up_2_")
    u3 = up_block(u2, c1, f, name="up_3_")
    
    outputs = Dense(class_num, activation = 'softmax')(u3)
    model = Model(inputs, outputs)
    
    return model

def ApplyUnet(trainX, trainY, testX, testY,TestTimes, activities, nb_epoch=100, fi=32):
    
    target_names,indices = get_activity_index(activities)
    #print(target_names)
    indices.append(len(target_names))
    target_names.append('EPadding')
    
    trainY_b = np_utils.to_categorical(trainY, len(target_names))
    testY_b = np_utils.to_categorical(testY, len(target_names))
    lb = LabelBinarizer()
    
    print(indices)
    lb.fit(indices)
    
    model = build_unet_model(trainX.shape[2],trainX.shape[3],len(target_names),f=fi)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
    model.summary()
    
    model_checkpoint = keras.callbacks.ModelCheckpoint('biounet-2018_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
    history = model.fit(trainX, trainY_b, epochs=nb_epoch, validation_split=0.2,callbacks=[model_checkpoint], verbose=0)
    
    fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
    
    tr_plt.plot(history.history['loss'])
    tr_plt.plot(history.history['val_loss'])
    tr_plt.set_title('Model loss')
    tr_plt.set_ylabel('Loss')
    tr_plt.set_xlabel('Epoch')
    tr_plt.legend(['loss', 'val_loss'], loc='upper left')

      
    predict_v = model.predict(testX)
    predict_vec = predict_v[0][0]
    confidences = np.max(predict_vec, axis=1)
    testYY = testY[0][0]
    
    predict_class = np.argmax(predict_vec, axis=1)
    predict = [lb.classes_[x] for x in predict_class]
    
    print("len(testYY)",len(testYY))
    print("len(predict)",len(predict))
    
    testXX = testX[0][0]
    

    
    for i in range(len(testYY)):
        #print(testYY[i])
        if testYY[i]==(len(target_names)-1):
            testYYY = testYY[:i]
            predict1 = predict[:i]
            testXXX = testXX[:i]
            ttimes = TestTimes[:i]
            Confidences = confidences[:i]
            print(testYY[i])
            print(i)
            break
        else:
            testYYY = testYY
            predict1 = predict
            testXXX = testXX
            ttimes = TestTimes
            Confidences = confidences

              
    print("len(testYYY)",len(testYYY))
    print("len(predict1)",len(predict1))
    
    result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
    print(result)
    testXX=testX
    cnf_matrix = confusion_matrix(testYYY, predict1)
    plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
    fig.tight_layout()
    fig.show()
    
    Sloss = history.history['loss']

    return predict1, testYYY, testXXX, Confidences, ttimes, Sloss

# Run leave_one_bird_out Cross Validation

In [5]:
def leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=200, fi=10):

    data_df.drop(['year'],axis=1,inplace=True)
    data_df = data_df[data_df['label']!='unlabeled']
    birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])

    for bird in birds:
        trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn=bird)
        trainX , testX = timeseries_standardize(trainX, testX)
        predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=nb_epoch, fi=fi)
        test_data_with_conf_pred = pd.DataFrame(data = {'timestamp':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})
        save_fn = 'res_'+bird+'.csv'
        save_path = os.path.join(save_folder,save_fn)
        test_data_with_conf_pred.to_csv(save_path,index=False)

## Load Year data

In [6]:
data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\umineko\2018\l_2018_acc.csv")

## Load labels

In [7]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\U_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()

## Get results

In [8]:
save_folder = r"G:\JaimeMorales\Codes\omizunagidori\database\results\baselines\U-Net\U_2018"

In [9]:
leave_one_bird_out_cv(data_df,save_folder,simclass_d, nb_epoch=200, fi=10)

LB07  :  37581  >  0  ?  True
bird:  LB07 lenght:  37581
bird:  LB08 lenght:  1421
LB09  :  55535  >  37581  ?  True
bird:  LB09 lenght:  55535
bird:  LB10 lenght:  8521
bird:  LB11 lenght:  1416
bird:  LB12 lenght:  10590
LB13  :  79642  >  55535  ?  True
bird:  LB13 lenght:  79642
bird:  LB14 lenght:  48236
bird:  LB15 lenght:  9375
bird:  LB16 lenght:  12787
bird:  LB17 lenght:  28386
reqlen:  79872
X(shape):  (11, 1, 79872, 3)
Y(shape):  (11, 1, 79872)
times(shape):  (11, 79872)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 79872, 3  0           []                               
                                )]                                                                
                              

<IPython.core.display.Javascript object>

1/1 [==============================] - 1s 526ms/step
len(testYY) 79872
len(predict) 79872
len(testYYY) 79872
len(predict1) 79872
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.49      0.12      0.19     26095
             unknown       0.08      0.77      0.14      2338
       flying_active       0.66      0.87      0.75      2525
      flying_passive       0.16      0.61      0.26      1044
          stationary       0.00      0.00      0.00      5579
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       1.00      1.00      1.00     42291
       foraging_poss       0.00      0.00      0.00         0
     forgaing_insect       0.00      0.00      0.00         0
  

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 340ms/step
len(testYY) 79872
len(predict) 79872
len(testYYY) 79872
len(predict1) 79872
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.00      0.00      0.00         0
             unknown       1.00      0.02      0.03      1421
       flying_active       0.00      0.00      0.00         0
      flying_passive       0.00      0.00      0.00         0
          stationary       0.00      0.00      0.00         0
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       1.00      1.00      1.00     78451
       foraging_poss       0.00      0.00      0.00         0
     forgaing_insect       0.00      0.00      0.00         0
  

...

1/1 [==============================] - 0s 281ms/step
len(testYY) 79872
len(predict) 79872
len(testYYY) 79872
len(predict1) 79872
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.87      0.94      0.90      3453
             unknown       0.59      0.63      0.61      1993
       flying_active       0.84      0.99      0.91      2579
      flying_passive       0.12      0.05      0.07       497
          stationary       0.53      0.02      0.03       662
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       1.00      1.00      1.00     70497
       foraging_poss       0.00      0.00      0.00         0
     forgaing_insect       0.00      0.00      0.00         0
  

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 275ms/step
len(testYY) 79872
len(predict) 79872
len(testYYY) 79872
len(predict1) 79872
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.00      0.00      0.00         0
             unknown       0.10      0.01      0.02      1234
       flying_active       0.95      0.93      0.94     11553
      flying_passive       0.00      0.00      0.00         0
          stationary       0.00      0.00      0.00         0
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       1.00      1.00      1.00     67085
       foraging_poss       0.00      0.00      0.00         0
     forgaing_insect       0.00      0.00      0.00         0
  

<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 279ms/step
len(testYY) 79872
len(predict) 79872
len(testYYY) 79872
len(predict1) 79872
                      precision    recall  f1-score   support

           unlabeled       0.00      0.00      0.00         0
   ground_stationary       0.42      0.75      0.54      4591
             unknown       0.50      0.29      0.37      4783
       flying_active       0.79      0.98      0.88     12739
      flying_passive       0.25      0.19      0.22      1413
          stationary       0.24      0.02      0.03      4860
        bathing_poss       0.00      0.00      0.00         0
             bathing       0.00      0.00      0.00         0
            preening       0.00      0.00      0.00         0
foraging_insect_poss       0.00      0.00      0.00         0
       ground_active       1.00      1.00      1.00     51486
       foraging_poss       0.00      0.00      0.00         0
     forgaing_insect       0.00      0.00      0.00         0
  

# Run induvidual test

## Load Data

In [10]:
data_df = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2019\l_2019_acc.csv")
data_df

,animal_tag,timestamp,acc_x,acc_y,acc_z,label,l_val,year
0,C5,1566418381000,0.015625,-0.066406,0.207031,unlabeled,0,2019
1,C5,1566418381040,0.089844,-0.121094,0.441406,unlabeled,0,2019
2,C5,1566418381080,-0.187500,-0.183594,0.093750,unlabeled,0,2019
3,C5,1566418381100,0.000000,0.000000,0.000000,unlabeled,0,2019
4,C5,1566418381104,0.000000,0.000000,0.000000,unlabeled,0,2019
...,...,...,...,...,...,...,...,...
1454484,C5,1566465299800,-0.101563,-0.417969,1.234375,unlabeled,0,2019
1454485,C5,1566465299840,-0.054688,-0.093750,1.214844,unlabeled,0,2019
1454486,C5,1566465299880,-0.117188,-0.140625,1.394531,unlabeled,0,2019
1454487,C5,1566465299920,-0.125000,-0.222656,1.410156,unlabeled,0,2019


In [11]:
data_df.drop(['year'],axis=1,inplace=True)
data_df = data_df[data_df['label']!='unlabeled']

In [12]:
birds = list(data_df.drop_duplicates(subset=['animal_tag'],keep = 'first')['animal_tag'])
birds

['C5']

## Divide train Test

In [13]:
trainX,testX,trainY,testY,testtimes,selector = get_train_test_bird(data_df,trn='LB0x')

C5  :  30537  >  0  ?  True
bird:  C5 lenght:  30537
reqlen:  30720
X(shape):  (1, 1, 30720, 3)
Y(shape):  (1, 1, 30720)
times(shape):  (1, 30720)


UnboundLocalError: local variable 'selector' referenced before assignment

In [ ]:
print(selector)

In [ ]:
test_data = data_df[data_df['animal_tag']==birds[selector]]

In [ ]:
test_data

In [ ]:
print('trainX(shape): ',trainX.shape)
print('trainY(shape): ',trainY.shape)

print('testX(shape): ',testX.shape)
print('testY(shape): ',testY.shape)
print('testtimes(len): ',len(testtimes))

In [ ]:
trainX , testX = timeseries_standardize(trainX, testX)

## Network testing

In [ ]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\omizunagidori\database\labels\O_labels_df.csv')
simclass_d = simclass_df['label'].to_dict()
#simclass_d[len('simclass_d')]='pad_class'

In [ ]:
simclass_d

In [ ]:
predict1, testYYY, testXXX, Confidences, Ttimes, Sloss = ApplyUnet(trainX, trainY, testX, testY, testtimes, simclass_d, nb_epoch=200, fi=10)

In [ ]:
test_data_with_conf_pred = pd.DataFrame(data = {'time':Ttimes, 'l_val':testYYY, 'predicted_label':predict1,'confidence':Confidences})

In [ ]:
save_folder = r'G:\JaimeMorales\Codes\Biodata\database\results'
save_fn = 'res_'+birds[selector]+'.csv'
save_path = os.path.join(save_folder,save_fn)
save_path

In [ ]:
test_data_with_conf_pred.to_csv(save_path,index=False)

In [ ]:
tttimes = range(len(predict1))

In [ ]:
labels,indices = get_activity_index(simclass_d)
indices.append(11)
labels.append('EPadding')
figs = plt.figure('result',figsize=(10, 4))
axs = plt.subplot(111)
axs.plot(tttimes, predict1, color='r')
axs.plot(tttimes, testYYY, color='g')
axs.set_ylabel('Activity label')
axs.set_yticks(indices)
axs.set_yticklabels(labels)
figs.show()